# Implementación WCEP para el problema de la mochila (relajado para que sea lineal)

Paper: Counterfactual explanations for linear optimization


In [1]:
import gurobipy as gp
from gurobipy import GRB

## Forward problem

## WCEP para una instancia en particular

# WCEP manteniendo H fijo y variando la dimensionalidad de x

## WCEP manteniendo x fijo y variando la dimensionalidad del H